In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype

# Import data and run preprocessing, such as grouping values, removing rows with null values, 
data = pd.read_csv('../datasets/marketing_campaign.csv', sep='\t')
data.head()
len(data.index)

2240

In [25]:
## Group column values

# Kidhome and Teenhome could be grouped as Kids
data['Kids'] = data['Kidhome'] + data['Teenhome']

# Marital_Status has the unique values ['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']
# These could be grouped as ["In relationship", "Not in relationship"]
re_marital = {
    'Single': 'Not in relationship',
    'Together': 'In relationship',
    'Married': 'In relationship',
    'Divorced': 'Not in relationship',
    'Widow': 'Not in relationship',
    'Alone': 'Not in relationship',
    'Absurd': 'Not in relationship',
    'YOLO': 'Not in relationship'
}
data['Marital_Status'] = data['Marital_Status'].map(re_marital)

# Encode customer time as timestamp when they were first collected as customers
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Kids
0,5524,1957,Graduation,Not in relationship,58138.0,0,0,2012-04-09,58,635,...,0,0,0,0,0,0,3,11,1,0
1,2174,1954,Graduation,Not in relationship,46344.0,1,1,2014-08-03,38,11,...,0,0,0,0,0,0,3,11,0,2
2,4141,1965,Graduation,In relationship,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,0,0,3,11,0,0
3,6182,1984,Graduation,In relationship,26646.0,1,0,2014-10-02,26,11,...,0,0,0,0,0,0,3,11,0,1
4,5324,1981,PhD,In relationship,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,0,0,3,11,0,1


In [26]:
## Calculate new column values

# Can create variable Family_size, which takes relationship status and # kids
data['Family_size'] = 0

for i in range(len(data)):
    if data['Marital_Status'][i] == 'In relationship':
        data['Family_size'][i] = 2 + data['Kids'][i]
    else:
        data['Family_size'][i] = 1 + data['Kids'][i]

# We assume that the data was collected at 2014-12-07 since the latest entry is from 2014-12-06
data['Age'] = 2014 - data['Year_Birth']
data['Collected'] = '2014-12-07'
data['Collected'] = pd.to_datetime(data['Collected'])
data['Days_is_client'] = (data['Collected'] - data['Dt_Customer']).dt.days
data['ActiveDays'] = data['Days_is_client'] - data['Recency']
data.head()

/var/folders/zs/m6739hc50mq3f4g25bdk88m80000gn/T/ipykernel_90814/4202787016.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Family_size'][i] = 1 + data['Kids'][i]
/var/folders/zs/m6739hc50mq3f4g25bdk88m80000gn/T/ipykernel_90814/4202787016.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Family_size'][i] = 2 + data['Kids'][i]


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Complain,Z_CostContact,Z_Revenue,Response,Kids,Family_size,Age,Collected,Days_is_client,ActiveDays
0,5524,1957,Graduation,Not in relationship,58138.0,0,0,2012-04-09,58,635,...,0,3,11,1,0,1,57,2014-12-07,972,914
1,2174,1954,Graduation,Not in relationship,46344.0,1,1,2014-08-03,38,11,...,0,3,11,0,2,3,60,2014-12-07,126,88
2,4141,1965,Graduation,In relationship,71613.0,0,0,2013-08-21,26,426,...,0,3,11,0,0,2,49,2014-12-07,473,447
3,6182,1984,Graduation,In relationship,26646.0,1,0,2014-10-02,26,11,...,0,3,11,0,1,3,30,2014-12-07,66,40
4,5324,1981,PhD,In relationship,58293.0,1,0,2014-01-19,94,173,...,0,3,11,0,1,3,33,2014-12-07,322,228


In [27]:
## Additional purchase information
data['MntTotal'] = data['MntWines']+data['MntFruits']+data['MntMeatProducts']+data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds']
data['NumAllPurchases'] = data['NumWebPurchases']+data['NumCatalogPurchases']+data['NumStorePurchases']
data['AverageCheck'] = round((data['MntTotal'] / data['NumAllPurchases']), 1)
data['ShareDealsPurchases'] = round((data['NumDealsPurchases'] / data['NumAllPurchases']) * 100, 1)
data['TotalAcceptedCmp'] = data['AcceptedCmp1']+data['AcceptedCmp2']+data['AcceptedCmp3']+data['AcceptedCmp4']+data['AcceptedCmp5']+data['Response']
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Family_size,Age,Collected,Days_is_client,ActiveDays,MntTotal,NumAllPurchases,AverageCheck,ShareDealsPurchases,TotalAcceptedCmp
0,5524,1957,Graduation,Not in relationship,58138.0,0,0,2012-04-09,58,635,...,1,57,2014-12-07,972,914,1617,22,73.5,13.6,1
1,2174,1954,Graduation,Not in relationship,46344.0,1,1,2014-08-03,38,11,...,3,60,2014-12-07,126,88,27,4,6.8,50.0,0
2,4141,1965,Graduation,In relationship,71613.0,0,0,2013-08-21,26,426,...,2,49,2014-12-07,473,447,776,20,38.8,5.0,0
3,6182,1984,Graduation,In relationship,26646.0,1,0,2014-10-02,26,11,...,3,30,2014-12-07,66,40,53,6,8.8,33.3,0
4,5324,1981,PhD,In relationship,58293.0,1,0,2014-01-19,94,173,...,3,33,2014-12-07,322,228,422,14,30.1,35.7,0


In [28]:
# Drop some rows containing null values
data = data.query("NumAllPurchases != 0")

# Replace some null values with a mean
data['Income'].fillna(data['Income'].mean(), inplace = True)

# Remove some unnecessary columns
data.drop(['ID', 'Year_Birth', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue', 'Collected'], axis = 1, inplace = True)

In [30]:
# Find any outliers
from peirces_criterion import separate_outliers, stats

for y in data:
    # print("Column: ",y)
    series = data[y].sort_values()
    unique_values = series.nunique()
    if is_numeric_dtype(data[y]):
        # print("Stats: ", stats(series))
        peirces = separate_outliers(series)
        # print("Outliers: ", peirces['outliers'])

# Based on outliers, drop some rows

data = data[data.Income < 200000]
data = data[data.Age < 114]
data = data[data.AverageCheck < 1600]
data = data[data.ShareDealsPurchases < 1500]


Data size before 71296
Data size after 71296


,Education,Marital_Status,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,Kids,Family_size,Age,Days_is_client,ActiveDays,MntTotal,NumAllPurchases,AverageCheck,ShareDealsPurchases,TotalAcceptedCmp
0,Graduation,Not in relationship,58138.0,58,635,88,546,172,88,88,...,0,1,57,972,914,1617,22,73.5,13.6,1
1,Graduation,Not in relationship,46344.0,38,11,1,6,2,1,6,...,2,3,60,126,88,27,4,6.8,50.0,0
2,Graduation,In relationship,71613.0,26,426,49,127,111,21,42,...,0,2,49,473,447,776,20,38.8,5.0,0
